In [1]:
import numpy as np
import torch
import pandas as pd
from comet import download_model, load_from_checkpoint
from typing import List
import json

2022-07-24 06:36:01.652302: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-24 06:36:01.652326: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
def load_cands(fname):
    data = []
    with open(fname, 'r') as file:
        while True:
            line = file.readline()
            #print(line)
            # if line is empty
            # end of file is reached
            if not line or len(line)<3:
                break

            data.append(json.loads(line))
    return data

def process_cands(cand_data):
    refs = []
    hyps = []
    srcs = []
    clen = len(cand_data['scores'])
    # return refs, hyps, srcs
    return [cand_data['ref']]*clen, cand_data['cands'], [cand_data['inp']]*clen
        

def get_average_score(cand_data):
    scoresum = 0
    scoretot = 0
    for c in cand_data:
        scoresum+=sum(c['scores'])
        scoretot+=len(c['scores'])
    return scoresum/scoretot


#print(get_average_score(beam_cands))
#print(get_average_score(lat_cands))

In [4]:
# cache dir for cometqe model
cometqe_dir = "./cometqemodel"
cometqe_model = "wmt20-comet-qe-da"
cometmodel = "wmt20-comet-da"
batch_size = 64

def get_flat(hyp_lists, srcs):
    flat_hyps = [hyp for sent_hyps in hyp_lists for hyp in sent_hyps]
    dup_srcs = [src for src in srcs for _ in range(len(hyp_lists[0]))]
    return flat_hyps, dup_srcs

In [ ]:
# script to run entire pipeline from a candidate jsonl file

cometqe_path = download_model(cometqe_model, cometqe_dir)
model = load_from_checkpoint(cometqe_path)
cand_data = load_cands("./candoutputs/latoutputcandinit.jsonl")
rerank_info = get_reranked_cands(cand_data)
storererank = {}
storererank['data'] = rerank_info
f = open('latticererank.json', 'w')
json.dump(storererank, f)
del model
del cometqe_path
comet_path = download_model(cometmodel, "./cometmodel")
comet = load_from_checkpoint(comet_path)
rrinfo = comet_rerank_info(rerank_info)
del comet
del comet_path

In [8]:
from transformers import MBartForConditionalGeneration, MBartTokenizer

In [7]:
tokenizer = MBartTokenizer.from_pretrained("facebook/mbart-many-to-one-mmt", src_lang="en_XX", tgt_lang="ro_RO")
        

400

In [5]:
def debug_cand(ind):

    print(cand_data[ind]['ref'])
    for c in cand_data[ind]['cands']:
        print(c)
        
def find_common(c1, c2list):
    for c in c2list:
        if c['ref']==c1['ref']:
            return c
        
beam_cands = load_cands("./candoutputs/beam1en_de.jsonl")
lat_cands = load_cands("./candoutputs/latoutputde2.jsonl")

In [24]:
lens = []
for l in lat_cands:
    lens.append(len(l['cands']))
print(sum(lens)/len(lens))

37.84158415841584


In [45]:
def is_different(lis):
    if len(lis)==0:
        return True
    prev = lis[0]
    for l in lis:
        if l is not prev:
            return True
        prev = l
    return False


In [ ]:
#print(beam_cands[0]['cands'][0])
def comp_debug_cand(ind, k):
    if len(lat_cands[ind]['cands'])==0:
        return
    print("INPUT")
    print(lat_cands[ind]['inp'])
    print("REF")
    print(lat_cands[ind]['ref'])
    print("LATTICE")
    for i in range(0, k):
        try:
            print(lat_cands[ind]['scores'][i])
            print(lat_cands[ind]['cands'][i])
        except:
            #print("NONE")
            ""
    print("BEAM")
    common = find_common(lat_cands[ind], beam_cands)
    #print(common)
    for i in range(0, k):
        print(common['scores'][i])
        print(common['cands'][i])
    print("BEAM")
    print(get_comet_scores([common['cands'][0]], [lat_cands[ind]['inp']], [lat_cands[ind]['ref']]))
    print("LATTICE")
    print(get_comet_scores([lat_cands[ind]['cands'][0]], [lat_cands[ind]['inp']], [lat_cands[ind]['ref']]))

for i in range(1, 20):
    comp_debug_cand(i, 5)

In [14]:
del cometqe_path
del model
del comet_path
del comet

NameError: name 'cometqe_path' is not defined

In [7]:
#cometqe_path = download_model(cometqe_model, cometqe_dir)
#model = load_from_checkpoint(cometqe_path)
comet_path = download_model(cometmodel, "./cometmodel")
comet = load_from_checkpoint(comet_path)

wmt20-comet-da is already in cache.
Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Encoder model frozen.


In [8]:
load_data = True
allscoressaved = []
def get_reranked_cands(c_data):
    global allscores
    reranked_info = []
    
    if load_data:
        allhyps = []
        allsrcs = []
        
        for i in range(0, len(c_data)):
            r, h, s = process_cands(c_data[i])
            allhyps.extend(h)
            allsrcs.extend(s)

        allscores = get_cometqe_scores(allhyps, allsrcs)
    for i in range(0, len(c_data)):
        try:
            tmp = {}
            r, h, s = process_cands(c_data[i])
            tmp['ref'] = r[0]
            tmp['src'] = s[0]
            tmp['topinitial'] = h[0]
            #scoretmp = get_cometqe_scores(h, s)[0]
            #print(scoretmp)
            slen = len(c_data[i]['scores'])
            scoretmp = allscores[0][i*slen:slen*(i+1)]
            comind = scoretmp.index(max(scoretmp))
            tmp['reranked'] = h[comind]
            tmp['initialsco'] = scoretmp[0]
            tmp['rersco'] = scoretmp[comind]
            reranked_info.append(tmp)
        except:
            print("something weird happenedc")
    return reranked_info
        

In [54]:
len(allscores[0])/800

48.0

In [60]:
rerank_info = get_reranked_cands(cand_data)
storererank = {}
storererank['data'] = rerank_info
f = open('rerankdata.json', 'w')
json.dump(storererank, f)

In [9]:
rercomettmp = []
firstcomettmp = []
def comet_rerank_info(rrk_info):
    global rercomettmp
    global firstcomettmp
    tophyps = []
    rerhyps = []
    refs = []
    srcs = []
    for r in rrk_info:
        tophyps.append(r['topinitial'])
        rerhyps.append(r['reranked'])
        refs.append(r['ref'])
        srcs.append(r['src'])
    rer_comet_sco = get_comet_scores(rerhyps, srcs, refs)
    first_comet_sco = get_comet_scores(tophyps, srcs, refs)
    rercomettmp = rer_comet_sco
    firstcomettmp = first_comet_sco
    print(rer_comet_sco[1])
    print(first_comet_sco[1])
    return rer_comet_sco, first_comet_sco

In [ ]:
comet_rerank_info(rerank_info)

In [10]:
def get_cometqe_scores(hyps, srcs):
    cometqe_input = [{"src": src, "mt": mt} for src, mt in zip(srcs, hyps)]
    # sentence-level and corpus-level COMET
    outputs = model.predict(
        cometqe_input, batch_size=40, progress_bar=True
    )
    torch.cuda.empty_cache()
    return outputs

def get_comet_scores(hyps, srcs, refs):
    cometqe_input = [{"src": src, "mt": mt, "ref":ref} for src, mt, ref in zip(srcs, hyps, refs)]
    # sentence-level and corpus-level COMET
    outputs = comet.predict(
        cometqe_input, batch_size=40, progress_bar=True
    )
    torch.cuda.empty_cache()
    return outputs

def test_cometqe(hyp, src):
    cqe_input = [{'src':src, 'mt':hyp}]
    outputs = model.predict(
        cqe_input, batch_size=1, progress_bar=True
    )
    torch.cuda.empty_cache()
    return outputs

In [1]:
test_srcs = [
    "I am doing very well today",
]

test_hyps = [
    [
        "Heute geht es mir sehr gut", # should be good
        "Mir geht es heute sehr schlecht", # should be bad
    ], 
]

ft_hyps, ft_srcs = get_flat(test_hyps, test_srcs)
get_cometqe_scores(ft_hyps, ft_srcs)

NameError: name 'get_flat' is not defined

In [1]:
import csv
import pandas as pd

def get_sample_set (num):
    with open("translation_data/news-commentary-v15.de-en.tsv") as file:
        tsv_file = csv.reader(file, delimiter="\t")
        i = 0
        res = []
        for f in tsv_file:
            if i == num:
                break
            res.append(f)
            i = i+1
    tmpdf = pd.DataFrame(res)
    tmpdf['de'] = tmpdf[0]
    tmpdf['en'] = tmpdf[1]
    del tmpdf[0]
    del tmpdf[1]
    return tmpdf

data = get_sample_set(10000)

In [2]:

data.to_csv('./translation_data/en_de.csv', )

,de,en
0,Steigt Gold auf 10.000 Dollar?,"$10,000 Gold?"
1,"SAN FRANCISCO – Es war noch nie leicht, ein ra...",SAN FRANCISCO – It has never been easy to have...
2,In letzter Zeit allerdings ist dies schwierige...,"Lately, with gold prices up more than 300% ove..."
3,Erst letzten Dezember verfassten meine Kollege...,"Just last December, fellow economists Martin F..."
4,"Und es kam, wie es kommen musste.",Wouldn’t you know it?
...,...,...
9995,Following its targeted killing of Iran's secon...,Following its targeted killing of Iran's secon...
9996,And that shift would occur at a time of growin...,And that shift would occur at a time of growin...
9997,The targeted killing by the United States of o...,The targeted killing by the United States of o...
9998,"After all, Iran and the US have already been a...","After all, Iran and the US have already been a..."


In [39]:
# candidate generation code

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import json

device = "cuda:1" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained("facebook/mbart-large-50-one-to-many-mmt")
#model = AutoModelForSeq2SeqLM.from_pretrained("facebook/mbart-large-50-one-to-many-mmt")
#model = model.to(device)


In [47]:
toks = [16844,599,39186,147,5877,1843,54810,68,2028,87046,4249,434,1061,122,128826,72537,1067,75900,24064,122,165,13501,33,135555,68,18341,1248,32847,165,235267,250003,2]
toks.reverse()
tokenizer.batch_decode([toks])

['</s>de_DE Zunächst und vielleicht vor allem die Revolutionen 1989 und der darauffolgende Zusammenbruch der Sowjetunion haben die globale Bipolarismus ein Ende']

In [3]:
#article_en = "The head of the United Nations says there is no military solution in Syria"


# translate from English to French (test)
"""
generated_tokens = model.generate(
    **model_inputs,
    forced_bos_token_id=tokenizer.lang_code_to_id["fr_XX"],
    do_sample=True, 
    max_length=50, 
    #top_k=50, 
    top_p=0.95, 
    num_return_sequences=10
    
)
"""
bs_cands = []
def get_bs_candidates(inputs, refs, per_example=10, batch=8):
    global bs_cands
    bs_cands = []
    tmpfile = open("./candoutputs/beamsearchcand"+str(per_example)+".jsonl", "w")
    for i in range(0, int(len(inputs)/batch)):
        print(i*batch)
        ins = inputs[i*batch:batch*(i+1)]
        ref = refs[i*batch:batch*(i+1)]
        model_inputs = tokenizer(ins, return_tensors="pt", truncation=True, padding=True).to(device)
        generated_tokens = model.generate(
            **model_inputs,
            forced_bos_token_id=tokenizer.lang_code_to_id["de_DE"],
            num_beams=per_example,
            num_return_sequences=per_example,
            return_dict_in_generate=True,
            output_scores=True,

        )
        allsco = generated_tokens.sequences_scores
        allcands = tokenizer.batch_decode(generated_tokens.sequences, skip_special_tokens=True)
        for j in range(0, batch):
            tmp = {}
            tmp['scores'] = list(allsco[j*per_example:per_example*(j+1)])
            tmp['scores'] = [tensor.item() for tensor in tmp['scores']]
            tmp['cands'] = list(allcands[j*per_example:per_example*(j+1)])
            tmp['inp'] = inputs[i*batch+j]
            tmp['ref'] = refs[i*batch+j]
            #print(tmp)
            bs_cands.append(tmp)
            tmpfile.write(json.dumps(tmp))
            tmpfile.write('\n')
            
        # ensure space is cleared
        del model_inputs
        del generated_tokens
        del allsco
        del allcands
    tmpfile.close()
    return bs_cands

nuc_cands = []
def get_nucleus_candidates(inputs, refs, per_example=10, batch=8):
    global nuc_cands
    nuc_cands = []
    tmpfile = open("./candoutputs/nucleuscand"+str(per_example)+".jsonl", "w")
    for i in range(0, int(len(inputs)/batch)):
        print(i*batch)
        ins = inputs[i*batch:batch*(i+1)]
        ref = refs[i*batch:batch*(i+1)]
        model_inputs = tokenizer(ins, return_tensors="pt", truncation=True, padding=True).to(device)
        generated_tokens = model.generate(
            **model_inputs,
            forced_bos_token_id=tokenizer.lang_code_to_id["de_DE"],
            do_sample=True, 
            max_length=50, 
            top_k=50, 
            top_p=0.6, 
            num_return_sequences=per_example,
            return_dict_in_generate=True,
            output_scores=True,

        )
        torch.cuda.empty_cache()
        allsco = generated_tokens.sequences_scores
        allcands = tokenizer.batch_decode(generated_tokens.sequences, skip_special_tokens=True)
        for j in range(0, batch):
            tmp = {}
            tmp['scores'] = list(allsco[j*per_example:per_example*(j+1)])
            tmp['scores'] = [tensor.item() for tensor in tmp['scores']]
            tmp['cands'] = list(allcands[j*per_example:per_example*(j+1)])
            tmp['inp'] = inputs[i*batch+j]
            tmp['ref'] = refs[i*batch+j]
            #print(tmp)
            nuc_cands.append(tmp)
            tmpfile.write(json.dumps(tmp))
            tmpfile.write('\n')
            
        # ensure space is cleared
        del model_inputs
        del generated_tokens
        del allsco
        del allcands
    tmpfile.close()
    return nuc_cands
        

In [4]:
torch.cuda.empty_cache()

In [ ]:
get_nucleus_candidates(list(data['en'][:800]), list(data['de'][:800]), 40, 2)
del nuc_cands
get_nucleus_candidates(list(data['en'][:800]), list(data['de'][:800]), 50, 1)
del nuc_cands
get_bs_candidates(list(data['en'][:800]), list(data['de'][:800]), 10, 2)
del bs_cands


#print(generated_tokens)
#print(len(generated_tokens.sequences_scores))
#tokenizer.batch_decode(generated_tokens.sequences, skip_special_tokens=True)

In [6]:
import json

In [8]:
del model

In [12]:
file1 = open("myfile.txt", "w")

In [15]:
testd = {'yo':"what is good buddy"}

file1.write(json.dumps(testd))
file1.write('\n')

1

In [16]:
file1.close()